In [13]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
import PIL
import numpy as np
import pandas as pd
import json

In [14]:
dir = '../data/train/'
train_file = 'data/train.json'
test_file = 'data/test.json'
train_img_file = 'data/train_regions.csv'
test_img_file = 'data/test_regions.csv'

In [15]:
# pretrenirani faster rcnn uz pomoc kog ce se izvlaciti regioni slika
def get_faster_rcnn():
    model = models.detection.fasterrcnn_resnet50_fpn(
        pretrained=True, 
        pretrained_backbone=True, 
        trainable_backbone_layers=0)
    model.eval()
    return model

In [16]:
def get_images(image_names, dir):
    images = []
    for image_name in image_names:
        image_path = dir + image_name
        image = PIL.Image.open(image_path)
        image = np.array(image)
        transform = transforms.Compose([transforms.ToTensor()])
        tensor = transform(image)
        images.append(tensor)
    # prvi element uredjenog para je lista tenzora (transformisanih slika), dok je
    # drugi element lista naziva tih slika
    return (images, image_names)

In [17]:
def get_regions(images):
    # images je uredjeni par, prvo su slike, a onda nazivi
    image_titles = images[1]
    images = images[0]
    # mapa gde je kljuc naziv slike, a vrednost lista regiona, tako da je svaki region
    # tenzor sa cetiri vrednosti (x1, x2, y1, y2) (?)
    images_regions = {}
    model = get_faster_rcnn()
    predictions = model(images)
    for imgindex, prediction in enumerate(predictions):
        title = image_titles[imgindex]
        images_regions[title] = []
        boxes = prediction['boxes']
        scores = prediction['scores']
        for regindex, score in enumerate(scores):
            # score se moze i menjati
            if score >= 0.5:
                images_regions[title].append(boxes[regindex].detach().numpy())
        print(title + ': ', len(images_regions[title]), ' regions')
    # ret - numpy niz [x1, y1, x2, y2]
    return images_regions

In [18]:
def get_feature_vectors(image_regions):
    images_feature_vectors = {}
    for title in image_regions.keys():
        image_path = dir + title
        image = PIL.Image.open(image_path)
        feature_vectors = []
        for region in image_regions[title]:
            x1 = region[0]
            y1 = region[1]
            x2 = region[2]
            y2 = region[3]
            region_image = image.crop((x1, y1, x2, y2))
            # oblik feature vektora - [1 1 1 .... 1 1] - duzine 512
            feature_vectors.append(get_feature_vector(region_image))
        images_feature_vectors[title] = feature_vectors
    # ret - mapa gde je kljuc naziv slike, a vrednost lista feature vektora
    return images_feature_vectors

In [19]:
def get_image_feature_vector(image_path):
    image = PIL.Image.open(image_path)
    return get_feature_vector(image)

In [20]:
def get_feature_vector(image):
    model = models.resnet18(pretrained = True)
    layer = model._modules.get('avgpool')
    model.eval()
    
    scaler = transforms.Resize((224, 224))
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
    to_tensor = transforms.ToTensor()
    
    # 2. Create a PyTorch Variable with the transformed image
    t_img = Variable(normalize(to_tensor(scaler(image))).unsqueeze(0))
    # 3. Create a vector of zeros that will hold our feature vector
    #    The 'avgpool' layer has an output size of 512
    my_embedding = None
    # 4. Define a function that will copy the output of a layer
    def my_hook(module_, input_, output_):
        nonlocal my_embedding
        my_embedding = output_
    # 5. Attach that function to our selected layer
    h = layer.register_forward_hook(my_hook)
    # 6. Run the model on our transformed image
    model(t_img)
    # 7. Detach our copy function from the layer
    h.remove()
    # 8. Return the feature vector
    my_embedding = my_embedding.squeeze(2).squeeze(2).squeeze(0)
    # ret - numpy niz duzine 512
    return my_embedding.detach().numpy()

In [21]:
# TUYNO I GLUPO - IZMENITI

def fvs2str(fvs):
    fvs = fvs[0]
    fvstr = ''
    last = len(fvs)-1
    for ifv, fv in enumerate(fvs):
        lastfv = len(fv) - 1
        for iv, v in enumerate(fv):
            fvstr += str(v)
            if not lastfv == iv:
                fvstr += '|'
        if not last == ifv:
            fvstr += '#'
    return fvstr

def str2fvs(string):
    fvs = []
    for strfv in string.split('#'):
        fv = []
        for v in strfv.split('|'):
            fv.append(float(v))
        fvs.append(fv)
    return fvs

In [36]:
def f(x):
    return x.tolist()

def f2(x):
    return list(map(f, x))

def preprocess_images(src_file, dest_file, img_dir):
    data = pd.read_json(src_file)
    image_names = data.image.values
    for i, image_name in enumerate(image_names):
        images_feature_vectors = get_feature_vectors(get_regions(get_images([image_name], img_dir)))
        dest_data = pd.DataFrame()
        dest_data['image'] = images_feature_vectors.keys()
        fvs = fvs2str(list(map(f2, list(images_feature_vectors.values()))))
        dest_data['vectors'] = fvs
        dest_data.to_csv(dest_file, mode='a', header=(i==0), index=False)
        
def preprocess_images2(image_names, dest_file, img_dir):
    for i, image_name in enumerate(image_names):
        image_feature_vector = get_image_feature_vector(img_dir + image_name)
        dest_data = pd.DataFrame()
        dest_data['image'] = [image_name]
        fvs = fvs2str([[image_feature_vector.tolist()]])
        dest_data['vectors'] = fvs
        dest_data.to_csv(dest_file, mode='a', header=(i==0), index=False)

### Formiranje feature vektora za regione slika iz trening skupa

In [174]:
preprocess_images(train_file, train_img_file, dir)

VizWiz_train_00001616.jpg:  2  regions
VizWiz_train_00011910.jpg:  2  regions
VizWiz_train_00013392.jpg:  0  regions
VizWiz_train_00007274.jpg:  1  regions
VizWiz_train_00015488.jpg:  2  regions
VizWiz_train_00002479.jpg:  2  regions
VizWiz_train_00016197.jpg:  2  regions
VizWiz_train_00010715.jpg:  0  regions
VizWiz_train_00006658.jpg:  0  regions
VizWiz_train_00017137.jpg:  2  regions
VizWiz_train_00004552.jpg:  20  regions
VizWiz_train_00010422.jpg:  0  regions
VizWiz_train_00001260.jpg:  1  regions
VizWiz_train_00007227.jpg:  0  regions
VizWiz_train_00014809.jpg:  4  regions
VizWiz_train_00012042.jpg:  0  regions
VizWiz_train_00008808.jpg:  1  regions
VizWiz_train_00009956.jpg:  5  regions
VizWiz_train_00013580.jpg:  4  regions
VizWiz_train_00019068.jpg:  0  regions
VizWiz_train_00019561.jpg:  5  regions
VizWiz_train_00000352.jpg:  2  regions
VizWiz_train_00016629.jpg:  2  regions
VizWiz_train_00016558.jpg:  0  regions
VizWiz_train_00013528.jpg:  1  regions
VizWiz_train_00000270.jp

VizWiz_train_00007368.jpg:  0  regions
VizWiz_train_00003702.jpg:  3  regions
VizWiz_train_00000090.jpg:  4  regions
VizWiz_train_00015096.jpg:  7  regions
VizWiz_train_00012749.jpg:  4  regions
VizWiz_train_00011255.jpg:  2  regions
VizWiz_train_00005158.jpg:  0  regions
VizWiz_train_00014086.jpg:  1  regions
VizWiz_train_00012094.jpg:  0  regions
VizWiz_train_00014078.jpg:  1  regions
VizWiz_train_00003193.jpg:  0  regions
VizWiz_train_00008291.jpg:  1  regions
VizWiz_train_00009898.jpg:  0  regions
VizWiz_train_00010104.jpg:  0  regions
VizWiz_train_00007661.jpg:  0  regions
VizWiz_train_00003735.jpg:  1  regions
VizWiz_train_00000382.jpg:  1  regions
VizWiz_train_00006727.jpg:  8  regions
VizWiz_train_00017499.jpg:  5  regions
VizWiz_train_00007141.jpg:  0  regions
VizWiz_train_00010087.jpg:  0  regions
VizWiz_train_00003332.jpg:  4  regions
VizWiz_train_00015151.jpg:  2  regions
VizWiz_train_00016323.jpg:  1  regions
VizWiz_train_00011197.jpg:  8  regions
VizWiz_train_00012564.jpg

VizWiz_train_00010595.jpg:  3  regions
VizWiz_train_00017230.jpg:  2  regions
VizWiz_train_00011177.jpg:  6  regions
VizWiz_train_00000642.jpg:  3  regions
VizWiz_train_00006498.jpg:  0  regions
VizWiz_train_00018915.jpg:  0  regions
VizWiz_train_00017581.jpg:  3  regions
VizWiz_train_00006893.jpg:  4  regions
VizWiz_train_00010255.jpg:  0  regions
VizWiz_train_00014966.jpg:  0  regions
VizWiz_train_00011315.jpg:  2  regions
VizWiz_train_00005317.jpg:  10  regions
VizWiz_train_00003768.jpg:  3  regions
VizWiz_train_00005720.jpg:  7  regions
VizWiz_train_00017858.jpg:  2  regions
VizWiz_train_00000594.jpg:  0  regions
VizWiz_train_00012615.jpg:  0  regions
VizWiz_train_00002509.jpg:  4  regions
VizWiz_train_00008717.jpg:  2  regions
VizWiz_train_00009087.jpg:  3  regions
VizWiz_train_00010774.jpg:  2  regions
VizWiz_train_00004908.jpg:  7  regions
VizWiz_train_00018603.jpg:  2  regions
VizWiz_train_00002673.jpg:  2  regions
VizWiz_train_00006729.jpg:  0  regions
VizWiz_train_00007684.jp

VizWiz_train_00011531.jpg:  1  regions
VizWiz_train_00001677.jpg:  8  regions
VizWiz_train_00012954.jpg:  6  regions
VizWiz_train_00018754.jpg:  0  regions
VizWiz_train_00000919.jpg:  0  regions
VizWiz_train_00008734.jpg:  2  regions
VizWiz_train_00012578.jpg:  0  regions
VizWiz_train_00005478.jpg:  3  regions
VizWiz_train_00017940.jpg:  5  regions
VizWiz_train_00014142.jpg:  1  regions
VizWiz_train_00007290.jpg:  0  regions
VizWiz_train_00014401.jpg:  3  regions
VizWiz_train_00001170.jpg:  1  regions
VizWiz_train_00008336.jpg:  3  regions
VizWiz_train_00004535.jpg:  3  regions
VizWiz_train_00006124.jpg:  3  regions
VizWiz_train_00008905.jpg:  1  regions
VizWiz_train_00009892.jpg:  0  regions
VizWiz_train_00013201.jpg:  2  regions
VizWiz_train_00002801.jpg:  7  regions
VizWiz_train_00015635.jpg:  5  regions
VizWiz_train_00012296.jpg:  1  regions
VizWiz_train_00015972.jpg:  6  regions
VizWiz_train_00000254.jpg:  0  regions
VizWiz_train_00005622.jpg:  5  regions
VizWiz_train_00001027.jpg

VizWiz_train_00019841.jpg:  2  regions
VizWiz_train_00019404.jpg:  1  regions
VizWiz_train_00007174.jpg:  4  regions
VizWiz_train_00018318.jpg:  1  regions
VizWiz_train_00013953.jpg:  4  regions
VizWiz_train_00008484.jpg:  1  regions
VizWiz_train_00016802.jpg:  5  regions
VizWiz_train_00006636.jpg:  2  regions
VizWiz_train_00018810.jpg:  0  regions
VizWiz_train_00011433.jpg:  1  regions
VizWiz_train_00018596.jpg:  0  regions
VizWiz_train_00006270.jpg:  3  regions
VizWiz_train_00012143.jpg:  0  regions
VizWiz_train_00001922.jpg:  5  regions
VizWiz_train_00001664.jpg:  2  regions
VizWiz_train_00019205.jpg:  0  regions
VizWiz_train_00002676.jpg:  0  regions
VizWiz_train_00011718.jpg:  1  regions
VizWiz_train_00019584.jpg:  2  regions
VizWiz_train_00005019.jpg:  0  regions
VizWiz_train_00018529.jpg:  1  regions
VizWiz_train_00008358.jpg:  0  regions
VizWiz_train_00016150.jpg:  0  regions
VizWiz_train_00019055.jpg:  0  regions
VizWiz_train_00019547.jpg:  2  regions
VizWiz_train_00008528.jpg

...

VizWiz_train_00002255.jpg:  6  regions
VizWiz_train_00017495.jpg:  2  regions
VizWiz_train_00003723.jpg:  0  regions
VizWiz_train_00012060.jpg:  0  regions
VizWiz_train_00000405.jpg:  0  regions
VizWiz_train_00010699.jpg:  0  regions
VizWiz_train_00011914.jpg:  2  regions
VizWiz_train_00013404.jpg:  0  regions
VizWiz_train_00005602.jpg:  1  regions
VizWiz_train_00005130.jpg:  0  regions
VizWiz_train_00006157.jpg:  0  regions
VizWiz_train_00015983.jpg:  2  regions
VizWiz_train_00006092.jpg:  4  regions
VizWiz_train_00016936.jpg:  1  regions
VizWiz_train_00004239.jpg:  14  regions
VizWiz_train_00018182.jpg:  2  regions
VizWiz_train_00010750.jpg:  5  regions
VizWiz_train_00014162.jpg:  1  regions
VizWiz_train_00001174.jpg:  4  regions
VizWiz_train_00016216.jpg:  5  regions
VizWiz_train_00008103.jpg:  6  regions
VizWiz_train_00013921.jpg:  6  regions
VizWiz_train_00008811.jpg:  0  regions
VizWiz_train_00012285.jpg:  8  regions
VizWiz_train_00006725.jpg:  1  regions
VizWiz_train_00010220.jp

VizWiz_train_00000469.jpg:  1  regions
VizWiz_train_00001546.jpg:  4  regions
VizWiz_train_00006834.jpg:  1  regions
VizWiz_train_00003544.jpg:  2  regions
VizWiz_train_00005927.jpg:  1  regions
VizWiz_train_00019120.jpg:  2  regions
VizWiz_train_00005419.jpg:  3  regions
VizWiz_train_00016116.jpg:  2  regions
VizWiz_train_00015479.jpg:  7  regions
VizWiz_train_00008676.jpg:  0  regions
VizWiz_train_00017339.jpg:  3  regions
VizWiz_train_00006475.jpg:  0  regions
VizWiz_train_00003275.jpg:  0  regions
VizWiz_train_00009771.jpg:  1  regions
VizWiz_train_00009814.jpg:  9  regions
VizWiz_train_00015897.jpg:  6  regions
VizWiz_train_00007606.jpg:  0  regions
VizWiz_train_00006536.jpg:  1  regions
VizWiz_train_00019807.jpg:  3  regions
VizWiz_train_00012969.jpg:  1  regions
VizWiz_train_00000891.jpg:  1  regions
VizWiz_train_00019756.jpg:  11  regions
VizWiz_train_00000897.jpg:  4  regions
VizWiz_train_00008713.jpg:  0  regions
VizWiz_train_00015048.jpg:  1  regions
VizWiz_train_00003062.jp

VizWiz_train_00016689.jpg:  2  regions
VizWiz_train_00012206.jpg:  8  regions
VizWiz_train_00004502.jpg:  3  regions
VizWiz_train_00004466.jpg:  1  regions
VizWiz_train_00004850.jpg:  0  regions
VizWiz_train_00016955.jpg:  0  regions
VizWiz_train_00006683.jpg:  2  regions
VizWiz_train_00000699.jpg:  7  regions
VizWiz_train_00012535.jpg:  4  regions
VizWiz_train_00000807.jpg:  1  regions
VizWiz_train_00002471.jpg:  3  regions
VizWiz_train_00003939.jpg:  2  regions
VizWiz_train_00000313.jpg:  5  regions
VizWiz_train_00007223.jpg:  3  regions
VizWiz_train_00009084.jpg:  2  regions
VizWiz_train_00000534.jpg:  0  regions
VizWiz_train_00002957.jpg:  1  regions
VizWiz_train_00011743.jpg:  1  regions
VizWiz_train_00017571.jpg:  1  regions
VizWiz_train_00012112.jpg:  2  regions
VizWiz_train_00017301.jpg:  0  regions
VizWiz_train_00005060.jpg:  0  regions
VizWiz_train_00008461.jpg:  0  regions
VizWiz_train_00014779.jpg:  6  regions
VizWiz_train_00006877.jpg:  5  regions
VizWiz_train_00018388.jpg

VizWiz_train_00016020.jpg:  5  regions
VizWiz_train_00003650.jpg:  5  regions
VizWiz_train_00010243.jpg:  2  regions
VizWiz_train_00016701.jpg:  1  regions
VizWiz_train_00001203.jpg:  1  regions
VizWiz_train_00007563.jpg:  2  regions
VizWiz_train_00012805.jpg:  0  regions
VizWiz_train_00006863.jpg:  1  regions
VizWiz_train_00018556.jpg:  1  regions
VizWiz_train_00003874.jpg:  1  regions
VizWiz_train_00005154.jpg:  1  regions
VizWiz_train_00015042.jpg:  4  regions
VizWiz_train_00003109.jpg:  3  regions
VizWiz_train_00019152.jpg:  0  regions
VizWiz_train_00000105.jpg:  1  regions
VizWiz_train_00004716.jpg:  2  regions
VizWiz_train_00017029.jpg:  2  regions
VizWiz_train_00013605.jpg:  2  regions
VizWiz_train_00004712.jpg:  0  regions
VizWiz_train_00019279.jpg:  1  regions
VizWiz_train_00010506.jpg:  0  regions
VizWiz_train_00017973.jpg:  0  regions
VizWiz_train_00006829.jpg:  3  regions
VizWiz_train_00014521.jpg:  3  regions
VizWiz_train_00008723.jpg:  2  regions
VizWiz_train_00014905.jpg

VizWiz_train_00001831.jpg:  9  regions
VizWiz_train_00002849.jpg:  0  regions
VizWiz_train_00008627.jpg:  1  regions
VizWiz_train_00017046.jpg:  1  regions
VizWiz_train_00000499.jpg:  2  regions
VizWiz_train_00018053.jpg:  4  regions
VizWiz_train_00015823.jpg:  8  regions
VizWiz_train_00013323.jpg:  3  regions
VizWiz_train_00019253.jpg:  3  regions
VizWiz_train_00003420.jpg:  0  regions
VizWiz_train_00015175.jpg:  1  regions
VizWiz_train_00000636.jpg:  1  regions
VizWiz_train_00018874.jpg:  0  regions
VizWiz_train_00003105.jpg:  4  regions
VizWiz_train_00017347.jpg:  2  regions
VizWiz_train_00000545.jpg:  2  regions
VizWiz_train_00004626.jpg:  1  regions
VizWiz_train_00005458.jpg:  1  regions
VizWiz_train_00001520.jpg:  4  regions
VizWiz_train_00006125.jpg:  0  regions
VizWiz_train_00016852.jpg:  0  regions
VizWiz_train_00015150.jpg:  0  regions
VizWiz_train_00018777.jpg:  0  regions
VizWiz_train_00018470.jpg:  0  regions
VizWiz_train_00005238.jpg:  0  regions
VizWiz_train_00004870.jpg

In [175]:
df = pd.read_csv(train_img_file)
df.head()

,image,vectors
0,VizWiz_train_00001616.jpg,0.4848161041736603|0.8302670121192932|0.333211...
1,VizWiz_train_00011910.jpg,0.5611845254898071|0.39622652530670166|0.21795...
2,VizWiz_train_00013392.jpg,NaN
3,VizWiz_train_00007274.jpg,1.396734595298767|0.0643806904554367|0.0101861...
4,VizWiz_train_00015488.jpg,0.34761351346969604|1.1078996658325195|0.77606...


In [176]:
len(str2fvs(df.vectors.values[0]))

2

In [177]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5614 entries, 0 to 5613
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   image    5614 non-null   object
 1   vectors  4042 non-null   object
dtypes: object(2)
memory usage: 87.8+ KB


In [178]:
preprocess_images(test_file, test_img_file, dir)

VizWiz_train_00007824.jpg:  3  regions
VizWiz_train_00007288.jpg:  2  regions
VizWiz_train_00006489.jpg:  4  regions
VizWiz_train_00009376.jpg:  0  regions
VizWiz_train_00006591.jpg:  2  regions
VizWiz_train_00004660.jpg:  2  regions
VizWiz_train_00017701.jpg:  2  regions
VizWiz_train_00012220.jpg:  0  regions
VizWiz_train_00016226.jpg:  3  regions
VizWiz_train_00004732.jpg:  0  regions
VizWiz_train_00018406.jpg:  7  regions
VizWiz_train_00000689.jpg:  2  regions
VizWiz_train_00009120.jpg:  2  regions
VizWiz_train_00001803.jpg:  3  regions
VizWiz_train_00015363.jpg:  4  regions
VizWiz_train_00017918.jpg:  1  regions
VizWiz_train_00012499.jpg:  1  regions
VizWiz_train_00008674.jpg:  2  regions
VizWiz_train_00004471.jpg:  2  regions
VizWiz_train_00015211.jpg:  5  regions
VizWiz_train_00004330.jpg:  1  regions
VizWiz_train_00005895.jpg:  7  regions
VizWiz_train_00008997.jpg:  6  regions
VizWiz_train_00004607.jpg:  13  regions
VizWiz_train_00010753.jpg:  10  regions
VizWiz_train_00001238.j

VizWiz_train_00001092.jpg:  0  regions
VizWiz_train_00003199.jpg:  3  regions
VizWiz_train_00018239.jpg:  0  regions
VizWiz_train_00013760.jpg:  0  regions
VizWiz_train_00013341.jpg:  2  regions
VizWiz_train_00005197.jpg:  1  regions
VizWiz_train_00002657.jpg:  1  regions
VizWiz_train_00017272.jpg:  3  regions
VizWiz_train_00001281.jpg:  2  regions
VizWiz_train_00012950.jpg:  1  regions
VizWiz_train_00007836.jpg:  0  regions
VizWiz_train_00001890.jpg:  3  regions
VizWiz_train_00005775.jpg:  1  regions
VizWiz_train_00016115.jpg:  6  regions
VizWiz_train_00019671.jpg:  0  regions
VizWiz_train_00008765.jpg:  0  regions
VizWiz_train_00011701.jpg:  4  regions
VizWiz_train_00008549.jpg:  0  regions
VizWiz_train_00002261.jpg:  2  regions
VizWiz_train_00006424.jpg:  3  regions
VizWiz_train_00008825.jpg:  0  regions
VizWiz_train_00013055.jpg:  1  regions
VizWiz_train_00005971.jpg:  0  regions
VizWiz_train_00018764.jpg:  2  regions
VizWiz_train_00015711.jpg:  2  regions
VizWiz_train_00010773.jpg

VizWiz_train_00015435.jpg:  0  regions
VizWiz_train_00010891.jpg:  4  regions
VizWiz_train_00007757.jpg:  0  regions
VizWiz_train_00007608.jpg:  0  regions
VizWiz_train_00008589.jpg:  0  regions
VizWiz_train_00008902.jpg:  1  regions
VizWiz_train_00006041.jpg:  0  regions
VizWiz_train_00010387.jpg:  10  regions
VizWiz_train_00003882.jpg:  0  regions
VizWiz_train_00013716.jpg:  0  regions
VizWiz_train_00005643.jpg:  1  regions
VizWiz_train_00014406.jpg:  0  regions
VizWiz_train_00005270.jpg:  5  regions
VizWiz_train_00011589.jpg:  1  regions
VizWiz_train_00015380.jpg:  4  regions
VizWiz_train_00018974.jpg:  5  regions
VizWiz_train_00006920.jpg:  7  regions
VizWiz_train_00001889.jpg:  8  regions
VizWiz_train_00005101.jpg:  0  regions
VizWiz_train_00007674.jpg:  0  regions
VizWiz_train_00004344.jpg:  5  regions
VizWiz_train_00008289.jpg:  0  regions
VizWiz_train_00012118.jpg:  1  regions
VizWiz_train_00005276.jpg:  3  regions
VizWiz_train_00012771.jpg:  3  regions
VizWiz_train_00004475.jp

In [179]:
df = pd.read_csv(test_img_file)
df.head()

,image,vectors
0,VizWiz_train_00007824.jpg,0.19898295402526855|0.1904696673154831|0.34966...
1,VizWiz_train_00007288.jpg,0.5149837136268616|1.2104671001434326|1.168068...
2,VizWiz_train_00006489.jpg,0.28766393661499023|0.45064228773117065|0.5715...
3,VizWiz_train_00009376.jpg,NaN
4,VizWiz_train_00006591.jpg,0.6279891729354858|0.39521658420562744|0.40628...


In [180]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 624 entries, 0 to 623
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   image    624 non-null    object
 1   vectors  443 non-null    object
dtypes: object(2)
memory usage: 9.9+ KB


### Racunanje feature vektora za slike bez pronadjenih regiona - trening skup

In [24]:
df = pd.read_csv(train_img_file)
df = df[~df.vectors.notnull()]
df.head()

,image,vectors
2,VizWiz_train_00013392.jpg,NaN
7,VizWiz_train_00010715.jpg,NaN
8,VizWiz_train_00006658.jpg,NaN
11,VizWiz_train_00010422.jpg,NaN
13,VizWiz_train_00007227.jpg,NaN


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1572 entries, 2 to 5610
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   image    1572 non-null   object
 1   vectors  0 non-null      object
dtypes: object(2)
memory usage: 36.8+ KB


In [37]:
preprocess_images2(df.image.values, 'data/train_regions2.csv', dir)

In [43]:
df = pd.read_csv(train_img_file)
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4042 entries, 0 to 5613
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   image    4042 non-null   object
 1   vectors  4042 non-null   object
dtypes: object(2)
memory usage: 94.7+ KB


In [44]:
df2 = pd.read_csv('data/train_regions2.csv')
df = pd.concat([df, df2])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5614 entries, 0 to 1571
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   image    5614 non-null   object
 1   vectors  5614 non-null   object
dtypes: object(2)
memory usage: 131.6+ KB


In [46]:
df.image.nunique()

5614

In [47]:
df.to_csv(train_img_file)

### Racunanje feature vektora za slike bez pronadjenih regiona - test skup

In [48]:
df = pd.read_csv(test_img_file)
df = df[~df.vectors.notnull()]
df.head()

,image,vectors
3,VizWiz_train_00009376.jpg,NaN
7,VizWiz_train_00012220.jpg,NaN
9,VizWiz_train_00004732.jpg,NaN
25,VizWiz_train_00001238.jpg,NaN
26,VizWiz_train_00007575.jpg,NaN


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 181 entries, 3 to 614
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   image    181 non-null    object
 1   vectors  0 non-null      object
dtypes: object(2)
memory usage: 4.2+ KB


In [50]:
preprocess_images2(df.image.values, 'data/test_regions2.csv', dir)

In [51]:
df = pd.read_csv(test_img_file)
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 443 entries, 0 to 623
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   image    443 non-null    object
 1   vectors  443 non-null    object
dtypes: object(2)
memory usage: 10.4+ KB


In [52]:
df2 = pd.read_csv('data/test_regions2.csv')
df = pd.concat([df, df2])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 624 entries, 0 to 180
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   image    624 non-null    object
 1   vectors  624 non-null    object
dtypes: object(2)
memory usage: 14.6+ KB


In [53]:
df.image.nunique()

624

In [54]:
df.to_csv(test_img_file)